In [ ]:
%%javascript
function ClickConnect(){
  console.log("Working");
  document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = '/content/gdrive/MyDrive/crime_incident'

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
!pip install ipyleaflet
from ipyleaflet import Map
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
import os
!pip install geopandas
import geopandas as gpd
import matplotlib.pyplot as plt
!pip install geoplot
import geoplot as gplt
from folium.raster_layers import ImageOverlay
import IPython.display as display

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.7 MB 5.3 MB/s 
     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 1.6 MB 41.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 16.7 MB 45.9 MB/s 
     |████████████████████████████████| 6.3 MB 34.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.8 MB 7.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 40.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 21.5 MB/s 
  In

# 数据预处理

重点关注按地理位置划分的犯罪类型，以及不同时间段的犯罪频率。

In [ ]:
df = pd.read_csv(os.path.join(root_path, 'Crime_Incidents_in_2020.csv'))
df = df.drop(columns=['X', 'Y', 'CCN', 'BLOCK', 'XBLOCK', 'YBLOCK', 'ANC',
            'PSA', 'BID', 'START_DATE', 'END_DATE', 'OCTO_RECORD_ID'])
df['COUNT'] = 1
df.head(5)

,REPORT_DAT,SHIFT,METHOD,OFFENSE,WARD,DISTRICT,NEIGHBORHOOD_CLUSTER,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,OBJECTID,COUNT
0,2020/07/09 02:13:24+00,EVENING,OTHERS,THEFT/OTHER,8,7.0,Cluster 38,007409 3,7409.0,Precinct 116,38.843884,-76.978919,237164351,1
1,2020/05/19 04:00:00+00,MIDNIGHT,GUN,HOMICIDE,8,7.0,Cluster 37,007407 2,7407.0,Precinct 119,38.862253,-76.993181,237164363,1
2,2020/05/19 04:00:00+00,MIDNIGHT,GUN,HOMICIDE,8,7.0,Cluster 37,007407 2,7407.0,Precinct 119,38.862253,-76.993181,237164364,1
3,2020/01/17 02:39:05+00,EVENING,OTHERS,THEFT F/AUTO,2,2.0,Cluster 8,005802 5,5802.0,Precinct 129,38.901478,-77.023986,237170410,1
4,2020/01/28 05:14:39+00,MIDNIGHT,OTHERS,THEFT F/AUTO,2,3.0,Cluster 7,005001 2,5001.0,Precinct 16,38.911868,-77.030785,237170411,1


In [ ]:
df.shape

(27912, 14)

In [ ]:
df.isna().sum()

REPORT_DAT               0
SHIFT                    0
METHOD                   0
OFFENSE                  0
WARD                     0
DISTRICT                29
NEIGHBORHOOD_CLUSTER     0
BLOCK_GROUP              1
CENSUS_TRACT             1
VOTING_PRECINCT          0
LATITUDE                 0
LONGITUDE                0
OBJECTID                 0
COUNT                    0
dtype: int64

将时间段分为日期和具体时间两个特征列

In [ ]:
new = df['REPORT_DAT'].str.split(' ', n=1, expand=True)
time = new[1].str.split('+', n=1, expand=True)
df['DATE'] = new[0] 
df['TIME'] = time[0]
df

,REPORT_DAT,SHIFT,METHOD,OFFENSE,WARD,DISTRICT,NEIGHBORHOOD_CLUSTER,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,OBJECTID,COUNT,DATE,TIME
0,2020/07/09 02:13:24+00,EVENING,OTHERS,THEFT/OTHER,8,7.0,Cluster 38,007409 3,7409.0,Precinct 116,38.843884,-76.978919,237164351,1,2020/07/09,02:13:24
1,2020/05/19 04:00:00+00,MIDNIGHT,GUN,HOMICIDE,8,7.0,Cluster 37,007407 2,7407.0,Precinct 119,38.862253,-76.993181,237164363,1,2020/05/19,04:00:00
2,2020/05/19 04:00:00+00,MIDNIGHT,GUN,HOMICIDE,8,7.0,Cluster 37,007407 2,7407.0,Precinct 119,38.862253,-76.993181,237164364,1,2020/05/19,04:00:00
3,2020/01/17 02:39:05+00,EVENING,OTHERS,THEFT F/AUTO,2,2.0,Cluster 8,005802 5,5802.0,Precinct 129,38.901478,-77.023986,237170410,1,2020/01/17,02:39:05
4,2020/01/28 05:14:39+00,MIDNIGHT,OTHERS,THEFT F/AUTO,2,3.0,Cluster 7,005001 2,5001.0,Precinct 16,38.911868,-77.030785,237170411,1,2020/01/28,05:14:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27907,2020/12/26 16:24:52+00,DAY,OTHERS,THEFT/OTHER,2,3.0,Cluster 8,004802 2,4802.0,Precinct 18,38.904903,-77.019900,237455609,1,2020/12/26,16:24:52
27908,2020/12/22 14:22:38+00,DAY,OTHERS,THEFT F/AUTO,2,3.0,Cluster 6,004300 1,4300.0,Precinct 141,38.915931,-77.036506,237455610,1,2020/12/22,14:22:38
27909,2020/12/23 16:41:41+00,DAY,OTHERS,THEFT/OTHER,1,3.0,Cluster 1,003902 1,3902.0,Precinct 35,38.923239,-77.044226,237455615,1,2020/12/23,16:41:41
27910,2020/12/19 02:04:23+00,EVENING,OTHERS,THEFT F/AUTO,6,1.0,Cluster 9,006400 1,6400.0,Precinct 127,38.873808,-77.009083,237455616,1,2020/12/19,02:04:23


In [ ]:
df = df.drop(columns=["REPORT_DAT"])
df['DATE'] = pd.to_datetime(df['DATE'], format= '%Y/%m/%d')
df.head(5)

,SHIFT,METHOD,OFFENSE,WARD,DISTRICT,NEIGHBORHOOD_CLUSTER,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,OBJECTID,COUNT,DATE,TIME
0,EVENING,OTHERS,THEFT/OTHER,8,7.0,Cluster 38,007409 3,7409.0,Precinct 116,38.843884,-76.978919,237164351,1,2020-07-09,02:13:24
1,MIDNIGHT,GUN,HOMICIDE,8,7.0,Cluster 37,007407 2,7407.0,Precinct 119,38.862253,-76.993181,237164363,1,2020-05-19,04:00:00
2,MIDNIGHT,GUN,HOMICIDE,8,7.0,Cluster 37,007407 2,7407.0,Precinct 119,38.862253,-76.993181,237164364,1,2020-05-19,04:00:00
3,EVENING,OTHERS,THEFT F/AUTO,2,2.0,Cluster 8,005802 5,5802.0,Precinct 129,38.901478,-77.023986,237170410,1,2020-01-17,02:39:05
4,MIDNIGHT,OTHERS,THEFT F/AUTO,2,3.0,Cluster 7,005001 2,5001.0,Precinct 16,38.911868,-77.030785,237170411,1,2020-01-28,05:14:39


In [ ]:
display(df.isna().sum() / len(df) * 100)
display(df.shape)

SHIFT                   0.000000
METHOD                  0.000000
OFFENSE                 0.000000
WARD                    0.000000
DISTRICT                0.103898
NEIGHBORHOOD_CLUSTER    0.000000
BLOCK_GROUP             0.003583
CENSUS_TRACT            0.003583
VOTING_PRECINCT         0.000000
LATITUDE                0.000000
LONGITUDE               0.000000
OBJECTID                0.000000
COUNT                   0.000000
DATE                    0.000000
TIME                    0.000000
dtype: float64

(27912, 15)

In [ ]:
df = df.dropna(axis = 0, how = 'any')

In [ ]:
df.to_csv(os.path.join(root_path, 'Crime_data_final.csv'))

# 探索性数据分析

In [ ]:
df = pd.read_csv(os.path.join(root_path, 'Crime_data_final.csv'))

In [ ]:
df_f = df
df_f['DATE'] = df_f['DATE'].astype(str)

df_f['YEAR'] = df_f['DATE'].str.split('-').str[0]
df_f['MONTH'] = df_f['DATE'].str.split('-').str[1]
df_f['DAY'] = df_f['DATE'].str.split('-').str[2]

df_f['HOUR'] = df_f['TIME'].str.split(':').str[0]
df_f['MINUTE'] = df_f['TIME'].str.split(':').str[1]
df_f['SECOND'] = df_f['TIME'].str.split(':').str[2]

df_f['YEAR'] = df_f['YEAR'].astype(int)
df_f['MONTH'] = df_f['MONTH'].astype(int)
df_f['DAY'] = df_f['DAY'].astype(int)

df_f['HOUR'] = df_f['HOUR'].astype(int)
df_f['MINUTE'] = df_f['MINUTE'].astype(int)
df_f['SECOND'] = df_f['SECOND'].astype(int)

In [ ]:
df_f.head()

,Unnamed: 0,SHIFT,METHOD,OFFENSE,WARD,DISTRICT,NEIGHBORHOOD_CLUSTER,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,...,OBJECTID,COUNT,DATE,TIME,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND
0,0,EVENING,OTHERS,THEFT/OTHER,8,7.0,Cluster 38,007409 3,7409.0,Precinct 116,...,237164351,1,2020-07-09,02:13:24,2020,7,9,2,13,24
1,1,MIDNIGHT,GUN,HOMICIDE,8,7.0,Cluster 37,007407 2,7407.0,Precinct 119,...,237164363,1,2020-05-19,04:00:00,2020,5,19,4,0,0
2,2,MIDNIGHT,GUN,HOMICIDE,8,7.0,Cluster 37,007407 2,7407.0,Precinct 119,...,237164364,1,2020-05-19,04:00:00,2020,5,19,4,0,0
3,3,EVENING,OTHERS,THEFT F/AUTO,2,2.0,Cluster 8,005802 5,5802.0,Precinct 129,...,237170410,1,2020-01-17,02:39:05,2020,1,17,2,39,5
4,4,MIDNIGHT,OTHERS,THEFT F/AUTO,2,3.0,Cluster 7,005001 2,5001.0,Precinct 16,...,237170411,1,2020-01-28,05:14:39,2020,1,28,5,14,39


# 自适应核密度分析

In [ ]:
df_f2 = df_f[['LONGITUDE', 'LATITUDE']].reset_index()
df_f2.rename(columns = {'index':'ID', 'LONGITUDE':'x', 'LATITUDE':'y'}, inplace = True)
df_f2.to_csv(root_path+'/crime_data_2.csv', index=False)

In [ ]:
df_f2

,ID,x,y
0,0,-76.978919,38.843884
1,1,-76.993181,38.862253
2,2,-76.993181,38.862253
3,3,-77.023986,38.901478
4,4,-77.030785,38.911868
...,...,...,...
27877,27877,-77.019900,38.904903
27878,27878,-77.036506,38.915931
27879,27879,-77.044226,38.923239
27880,27880,-77.009083,38.873808


In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import time
def adaptiveKDE(input_dir,result_dir):
    lines=open(input_dir).readlines()
    result=open(result_dir,'w+')
    result.write('ID,x,y,h\n')
    time_start=time.time()
    a, delta_a, e_a=0.5, 0.1, 0.005
    points={}
    x=[]
    y=[]
    dis=[]
    for line in lines:
            attr = line.split("\n")[0].split(",")
            if attr[1] != 'x':
                if attr[0] not in points:
                        points[attr[0]]=[float(attr[1])]
                        points[attr[0]].append(float(attr[2]))
                        x.append(float(attr[1]))
                        y.append(float(attr[2]))

    #plt.plot(x,y,'bo')
    x=np.array(x)
    y=np.array(y)
    x_mean=np.mean(x)
    y_mean=np.mean(y)
    for i in range(len(x)):
        dis.append(math.pow(math.pow(x[i]-x_mean,2)+math.pow(y[i]-y_mean,2),0.5))
    dis_array=np.array(dis)
    q1=np.percentile(dis_array,25)
    q3=np.percentile(dis_array,75)
    dis_std=np.std(dis_array)
    n=len(x)
    
    delta=min(dis_std,(q3-q1)/1.34)
    h=1.06*delta*pow(len(x),-0.2)
    print("thumb-h:",h)
    delta_h=h/10
    e_h=delta_h/20
    iters=0
    max_iters=12
    while (delta_a>e_a or delta_h>e_h) and max_iters>iters:
        iters+=1
        g=[1,1,1]
        h_list=[h-delta_h,h,h+delta_h]
        for i in points:
            fi=[0,0,0]
            for j in points:
                dij_square=pow(points[i][0]-points[j][0],2)+pow(points[i][1]-points[j][1],2)
                for h_idx in range(len(h_list)):
                    h1=h_list[h_idx]
                    fi[h_idx]+=pow(math.e,-dij_square/(2*pow(h1,2)))
            for h_idx in range(len(h_list)):
                h1 = h_list[h_idx]
                fi[h_idx]=1/(2*math.pi*n*pow(h1,2))*fi[h_idx]
                if len(points[i])<10:
                    points[i].append(fi[h_idx])
                else:
                    points[i][2+h_idx]=fi[h_idx]
                if fi[h_idx]!=0:
                    g[h_idx]+=math.log(fi[h_idx])
        #x,y,fi(h-delta_h),fi(h),fi(h+delta_h)
        a_list=[a-delta_a,a,a+delta_a]
        for h_idx in range(len(h_list)):
            g[h_idx]=pow(math.e,g[h_idx]/n)
            for i in points:
                if h_idx==0:
                    a1=a_list[0]
                    hi=pow(points[i][2+h_idx]/g[h_idx],-a1)*h_list[h_idx]
                    if len(points[i]) < 10:
                        points[i].append(hi)
                    else:
                        points[i][5]=hi
                elif h_idx==1:
                    for a_idx in range(len(a_list)):
                        a1 = a_list[a_idx]
                        hi = pow(points[i][2 + h_idx] / g[h_idx], -a1) * h_list[h_idx]
                        if len(points[i]) < 10:
                            points[i].append(hi)
                        else:
                            points[i][6+a_idx] = hi
                else:
                    a1 = a_list[2]
                    hi = pow(points[i][2 + h_idx] / g[h_idx], -a1) * h_list[h_idx]
                    if len(points[i]) < 10:
                        points[i].append(hi)
                    else:
                        points[i][9] = hi
        #x,y,fi(h-delta_h),fi(h),fi(h+delta_h),hi(a-,h-),hi(a-,h),hi(a,h),hi(a+,h),hi(a+,h+)
        L=[0,0,0,0,0]
        for i in points:
            fi=np.zeros(5)
            for j in points:
                if i!=j:
                    dij_square=pow(points[i][0]-points[j][0],2)+pow(points[i][1]-points[j][1],2)
                    for fi_idx in range(len(fi)):
                        fi[fi_idx]+=pow(math.e,-dij_square/(2*pow(points[j][5+fi_idx],2)))/pow(points[j][5+fi_idx],2)
            for fi_idx in range(len(fi)):
                fi[fi_idx]=1/(2*math.pi*(n-1))*fi[fi_idx]
                if fi[fi_idx]!=0:
                    L[fi_idx]+=math.log(fi[fi_idx])
        max_L=max(L)
        if max_L==L[2]:
            delta_a=delta_a/2
            delta_h=delta_h/2
        else:
            idx=L.index(max_L)
            if idx==0:
                a=a-delta_a
                h=h-delta_h
            elif idx==1:
                a=a-delta_a
            elif idx==3:
                a=a+delta_a
            elif idx==4:
                a = a + delta_a
                h = h + delta_h
        print(h)
    time_end=time.time()
    print('totally cost',time_end-time_start)
    print("the parameter of h:"+str(h)+",the parameter of a:"+str(a))
    for i in points:
        line=i+','+str(points[i][0])+','+str(points[i][1])+','+str(points[i][7])+'\n'
        result.write(line)
    result.close()


In [ ]:
input_dir = root_path+'/crime_POI_1000.csv'
result_dir = root_path+'/adaptiveKDE_h_1000.csv'
adaptiveKDE(input_dir,result_dir)

thumb-h: 0.006120763390109116
0.005508687051098204
0.004896610712087292
0.00428453437307638
0.0036724580340654683
0.0030603816950545565
0.0030603816950545565
0.0030603816950545565
0.0024483053560436447
0.0024483053560436447
0.0024483053560436447
0.0024483053560436447
0.0024483053560436447
totally cost 162.86248064041138
the parameter of h:0.0024483053560436447,the parameter of a:0.5


In [ ]:
df_1000 = pd.read_csv(root_path+'/crime_data_2.csv', nrows=1000)
df_5000 = pd.read_csv(root_path+'/crime_data_2.csv', nrows=5000)
df_10000 = pd.read_csv(root_path+'/crime_data_2.csv', nrows=10000)
df_1000.to_csv(root_path+'/crime_POI_1000.csv', index=False)
df_5000.to_csv(root_path+'/crime_POI_5000.csv', index=False)
df_10000.to_csv(root_path+'/crime_POI_10000.csv', index=False)